In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm

In [2]:
data = pd.read_csv('/kaggle/input/house-price/house_prices.csv')

In [3]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187531 entries, 0 to 187530
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Index              187531 non-null  int64  
 1   Title              187531 non-null  object 
 2   Description        184508 non-null  object 
 3   Amount(in rupees)  187531 non-null  object 
 4   Price (in rupees)  169866 non-null  float64
 5   location           187531 non-null  object 
 6   Carpet Area        106858 non-null  object 
 7   Status             186916 non-null  object 
 8   Floor              180454 non-null  object 
 9   Transaction        187448 non-null  object 
 10  Furnishing         184634 non-null  object 
 11  facing             117298 non-null  object 
 12  overlooking        106095 non-null  object 
 13  Society            77853 non-null   object 
 14  Bathroom           186703 non-null  object 
 15  Balcony            138596 non-null  object 
 16  Ca

In [4]:
data.drop(columns=['Index','Dimensions','Plot Area'])

,Title,Description,Amount(in rupees),Price (in rupees),location,Carpet Area,Status,Floor,Transaction,Furnishing,facing,overlooking,Society,Bathroom,Balcony,Car Parking,Ownership,Super Area
0,1 BHK Ready to Occupy Flat for sale in Srushti...,"Bhiwandi, Thane has an attractive 1 BHK Flat f...",42 Lac,6000.0,thane,500 sqft,Ready to Move,10 out of 11,Resale,Unfurnished,NaN,NaN,Srushti Siddhi Mangal Murti Complex,1,2,NaN,NaN,NaN
1,2 BHK Ready to Occupy Flat for sale in Dosti V...,One can find this stunning 2 BHK flat for sale...,98 Lac,13799.0,thane,473 sqft,Ready to Move,3 out of 22,Resale,Semi-Furnished,East,Garden/Park,Dosti Vihar,2,NaN,1 Open,Freehold,NaN
2,2 BHK Ready to Occupy Flat for sale in Sunrise...,Up for immediate sale is a 2 BHK apartment in ...,1.40 Cr,17500.0,thane,779 sqft,Ready to Move,10 out of 29,Resale,Unfurnished,East,Garden/Park,Sunrise by Kalpataru,2,NaN,1 Covered,Freehold,NaN
3,1 BHK Ready to Occupy Flat for sale Kasheli,This beautiful 1 BHK Flat is available for sal...,25 Lac,NaN,thane,530 sqft,Ready to Move,1 out of 3,Resale,Unfurnished,NaN,NaN,NaN,1,1,NaN,NaN,NaN
4,2 BHK Ready to Occupy Flat for sale in TenX Ha...,"This lovely 2 BHK Flat in Pokhran Road, Thane ...",1.60 Cr,18824.0,thane,635 sqft,Ready to Move,20 out of 42,Resale,Unfurnished,West,"Garden/Park, Main Road",TenX Habitat Raymond Realty,2,NaN,1 Covered,Co-operative Society,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187526,3 BHK Ready to Occupy Flat for sale in Bollywo...,This magnificent 3 BHK Flat is available for s...,63 Lac,3225.0,zirakpur,NaN,Ready to Move,2 out of 4,New Property,Semi-Furnished,East,Garden/Park,Bollywood Esencia,3,3,1 Covered,Freehold,1953 sqft
187527,3 BHK Ready to Occupy Flat for sale in Sushma ...,Have a look at this immaculate 3 BHK flat for ...,55 Lac,3274.0,zirakpur,NaN,Ready to Move,4 out of 6,Resale,Unfurnished,North - East,"Garden/Park, Main Road",Sushma Urban Views,3,NaN,1 Covered,NaN,1680 sqft
187528,3 BHK Ready to Occupy Flat for sale in Bollywo...,"Gazipur, Zirakpur has an appealing 3 BHK flat ...",76 Lac,4343.0,zirakpur,1250 sqft,Ready to Move,1 out of 3,Resale,Furnished,East,"Garden/Park, Main Road",Bollywood Esencia,3,2,"1 Covered,",Freehold,NaN
187529,2 BHK Ready to Occupy Flat for sale in Friends...,Up for immediate sale is a 2 BHK apartment in ...,30 Lac,4231.0,zirakpur,NaN,Ready to Move,2 out of 2,Resale,Semi-Furnished,NaN,Main Road,Friends Enclave,2,NaN,NaN,NaN,709 sqft


In [5]:
data = data.dropna(subset=['Price (in rupees)'])
data = data.dropna(subset=['Carpet Area'])

In [7]:
import re

def convert_amount(amount):
    if pd.isnull(amount):
        return np.nan
    
    amount = amount.replace(',', '')  
    
    if 'Lac' in amount:
        value = float(amount.replace('Lac', '').strip())
        return value * 100000
    elif 'Cr' in amount:
        value = float(amount.replace('Cr', '').strip())
        return value * 10000000
    else:
        return float(amount)

data['Amount(in rupees)'] = data['Amount(in rupees)'].apply(convert_amount)

In [8]:
import re

units = data['Carpet Area'].str.extract(r'(\D+)$', expand=False).unique()
print(units)

[' sqft' ' sqm' ' sqyrd' ' acre' ' ground' ' bigha' ' marla' ' kanal']


In [9]:
import re

def convert_area_unit(area_str):
    area_str = str(area_str)
    units = {
        'sqft': 1.0,
        'sqm': 10.7639,
        'sqyd': 9.0,
        'acre': 43560.0,
        'ground': 2400.0,
        'bigha': 27225.0,
        'marla': 272.25,
        'kanal': 5445.0
    }
    
    match = re.search(r'(\d+(?:\.\d+)?)\s*(\w+)', area_str)
    if match:
        value, unit = match.groups()
        value = float(value)
        unit = unit.lower()
        if unit in units:
            return value * units[unit]
        else:
            return None
    else:
        return None

data['Carpet Area'] = data['Carpet Area'].apply(convert_area_unit)

In [10]:
sorted_data = data.sort_values(by=['Amount(in rupees)'], ascending=False)

In [11]:
top_100_houses = sorted_data.head(100)

In [12]:
top_100_houses

,Index,Title,Description,Amount(in rupees),Price (in rupees),location,Carpet Area,Status,Floor,Transaction,...,facing,overlooking,Society,Bathroom,Balcony,Car Parking,Ownership,Super Area,Dimensions,Plot Area
181234,181234,3 BHK Ready to Occupy Flat for sale in Nest Ha...,Have a look at this immaculate 3 BHK flat for ...,1.400300e+10,6700000.0,vadodara,1252.0,Ready to Move,5 out of 9,New Property,...,NaN,NaN,Nest Harmony,3,5,1 Covered,NaN,NaN,NaN,NaN
176536,176536,3 BHK Ready to Occupy Flat for sale Hill Cart ...,This ready to move-in 3 BHK flat is available ...,5.100400e+09,4041600.0,siliguri,970.0,Ready to Move,1 out of 4,New Property,...,North,Main Road,NaN,2,1,1 Covered,Freehold,NaN,NaN,NaN
174894,174894,2 BHK Ready to Occupy Flat for sale Kachna Road,"Kachna Road, Raipur has an attractive 2 BHK Fl...",3.967500e+09,3450000.0,raipur,920.0,Ready to Move,2 out of 2,New Property,...,East,"Garden/Park, Pool, Main Road",NaN,2,2,"1 Covered,",Freehold,NaN,NaN,NaN
175013,175013,2 BHK Ready to Occupy Flat for sale in Chaitan...,"This lovely 2 BHK Flat in Saddu, Raipur is up ...",2.298000e+09,2669100.0,raipur,750.0,Ready to Move,2 out of 9,New Property,...,East,"Garden/Park, Main Road",Chaitanya Greens,2,1,"1 Covered,",Freehold,NaN,NaN,NaN
50321,50321,5 BHK Ready to Occupy Flat for sale in DLF The...,Discover this immaculate 5 BHK flat for sale a...,5.500000e+08,52381.0,gurgaon,5500.0,Ready to Move,25 out of 25,Resale,...,North,Main Road,DLF The Magnolias,5,3,"3 Covered,",Freehold,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2246,2246,3 BHK Ready to Occupy Flat for sale in Lokhand...,"This lovely 3 BHK Flat in Mahalakshmi, Mumbai ...",1.320000e+08,61395.0,mumbai,1600.0,Ready to Move,44 out of 91,New Property,...,North - East,"Garden/Park, Pool",Lokhandwala Minerva,3,2,3 Covered,Freehold,NaN,NaN,NaN
114735,114735,3 BHK Ready to Occupy Flat for sale in Lokhand...,"This lovely 3 BHK Flat in Mahalakshmi, Mumbai ...",1.320000e+08,61395.0,mumbai,1600.0,Ready to Move,44 out of 91,New Property,...,North - East,"Garden/Park, Pool",Lokhandwala Minerva,3,2,3 Covered,Freehold,NaN,NaN,NaN
116891,116891,4 BHK Ready to Occupy Flat for sale Punjabi Ba...,This attractive 4 BHK apartment can be found f...,1.300000e+08,18759.0,new-delhi,NaN,Ready to Move,3 out of 4,New Property,...,North - West,Main Road,NaN,4,2,"3 Covered,",Freehold,NaN,NaN,NaN
160896,160896,5 BHK Ready to Occupy Flat for sale in Jaypee ...,This exquisite 5 BHK Flat is offered for sale ...,1.300000e+08,28889.0,greater-noida,NaN,Ready to Move,1 out of 3,Resale,...,NaN,"Garden/Park, Pool, Main Road",Jaypee Green Villas,7,3,5 Open,Leasehold,NaN,NaN,NaN


In [14]:
import plotly.express as px

location_counts = top_100_houses['location'].value_counts()

fig = px.bar(x=location_counts.index, y=location_counts.values, 
             labels={'x': 'Vị trí', 'y': 'Số lượng ngôi nhà'},
             title='Số lượng ngôi nhà đắt nhất theo vị trí')

fig.update_layout(xaxis_tickangle=-45, font=dict(size=12))

fig.show()

In [15]:
cities = ['vadodara', 'siliguri', 'raipur']
filtered_houses = top_100_houses[top_100_houses['location'].isin(cities)]

top_4_houses = filtered_houses.sort_values(by='Price (in rupees)', ascending=False).head(4)

top_4_houses

,Index,Title,Description,Amount(in rupees),Price (in rupees),location,Carpet Area,Status,Floor,Transaction,...,facing,overlooking,Society,Bathroom,Balcony,Car Parking,Ownership,Super Area,Dimensions,Plot Area
181234,181234,3 BHK Ready to Occupy Flat for sale in Nest Ha...,Have a look at this immaculate 3 BHK flat for ...,1.400300e+10,6700000.0,vadodara,1252.0,Ready to Move,5 out of 9,New Property,...,NaN,NaN,Nest Harmony,3,5,1 Covered,NaN,NaN,NaN,NaN
176536,176536,3 BHK Ready to Occupy Flat for sale Hill Cart ...,This ready to move-in 3 BHK flat is available ...,5.100400e+09,4041600.0,siliguri,970.0,Ready to Move,1 out of 4,New Property,...,North,Main Road,NaN,2,1,1 Covered,Freehold,NaN,NaN,NaN
174894,174894,2 BHK Ready to Occupy Flat for sale Kachna Road,"Kachna Road, Raipur has an attractive 2 BHK Fl...",3.967500e+09,3450000.0,raipur,920.0,Ready to Move,2 out of 2,New Property,...,East,"Garden/Park, Pool, Main Road",NaN,2,2,"1 Covered,",Freehold,NaN,NaN,NaN
175013,175013,2 BHK Ready to Occupy Flat for sale in Chaitan...,"This lovely 2 BHK Flat in Saddu, Raipur is up ...",2.298000e+09,2669100.0,raipur,750.0,Ready to Move,2 out of 9,New Property,...,East,"Garden/Park, Main Road",Chaitanya Greens,2,1,"1 Covered,",Freehold,NaN,NaN,NaN


In [16]:
cheap_100_houses = sorted_data.tail(100)

In [17]:
cheap_100_houses

,Index,Title,Description,Amount(in rupees),Price (in rupees),location,Carpet Area,Status,Floor,Transaction,...,facing,overlooking,Society,Bathroom,Balcony,Car Parking,Ownership,Super Area,Dimensions,Plot Area
146398,146398,1 BHK Ready to Occupy Flat for sale in Xrbia H...,Carefully laid out in the prime location of Ma...,1200000.0,3692.0,pune,208.0,Ready to Move,3 out of 8,Resale,...,NaN,Garden/Park,Xrbia Hinjewadi,1,1,1 Covered,Leasehold,NaN,NaN,NaN
154100,154100,2 BHK Ready to Occupy Flat for sale Durgapur,Up for immediate sale is a 2 BHK apartment in ...,1200000.0,2400.0,durgapur,400.0,Ready to Move,1 out of 2,Resale,...,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN
155007,155007,1 BHK Ready to Occupy Flat for sale Sector 76,Discover this immaculate 1 BHK flat for sale a...,1200000.0,3000.0,faridabad,400.0,Ready to Move,Ground out of 2,Resale,...,North,"Garden/Park, Main Road",NaN,1,NaN,1 Open,Freehold,NaN,NaN,NaN
91197,91197,1 BHK Ready to Occupy Flat for sale Uttarpara ...,Multistorey apartment is available for sale. I...,1200000.0,2817.0,kolkata,400.0,Ready to Move,Ground out of 4,Resale,...,NaN,NaN,NaN,1,NaN,NaN,Power Of Attorney,NaN,NaN,NaN
149680,149680,1 BHK Ready to Occupy Flat for sale in Krish C...,This exquisite 1 BHK Flat is offered for sale ...,1200000.0,2474.0,bhiwadi,485.0,Ready to Move,3 out of 14,Resale,...,NaN,NaN,Krish City Heights,1,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173053,173053,1 BHK Ready to Occupy Flat for sale Old Satpat...,1 BHK flat available for sale in Palghar in th...,200000.0,233.0,palghar,750.0,Ready to Move,2 out of 7,New Property,...,NaN,NaN,NaN,2,3,NaN,NaN,NaN,NaN,NaN
167186,167186,3 BHK Ready to Occupy Flat for sale Chikan Ghar,3 BHK flat available for sale in Kalyan in the...,140000.0,99.0,kalyan,1025.0,Ready to Move,9 out of 12,Resale,...,East,"Garden/Park, Main Road",NaN,3,2,NaN,Freehold,NaN,NaN,NaN
167653,167653,2 BHK Ready to Occupy Flat for sale Dombivli,This gorgeous 2 BHK Flat is available for sale...,140000.0,98.0,kalyan,1383.0,Ready to Move,4 out of 25,New Property,...,North,"Garden/Park, Main Road",NaN,3,NaN,NaN,Freehold,NaN,NaN,NaN
165547,165547,2 BHK Ready to Occupy Flat for sale Ghoramara,Up for immediate sale is a 2 BHK apartment in ...,120000.0,160.0,guwahati,875.0,Ready to Move,1 out of 2,New Property,...,North - East,NaN,NaN,2,1,NaN,Freehold,NaN,NaN,NaN


In [19]:
import plotly.express as px


location_counts = cheap_100_houses['location'].value_counts()


fig = px.bar(x=location_counts.index, y=location_counts.values, 
             labels={'x': 'Vị trí', 'y': 'Số lượng ngôi nhà'},
             title='Số lượng ngôi nhà nghèo nhất theo vị trí')

fig.update_layout(xaxis_tickangle=-45, font=dict(size=12))


fig.show()